In [14]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [15]:
xls = pd.ExcelFile('./EZStacking_config.ods', engine="odf")
meta_package = pd.read_excel(xls, 'meta_package')
package_source = pd.read_excel(xls, 'package_source')
package = pd.read_excel(xls, 'package')
document = pd.read_excel(xls, 'document')

In [29]:
package_source_type = 'full'
data_size = 'large'
kaggle = True

In [30]:
pd_pk_import = package_source[(package_source.package_source_type   == package_source_type) &\
                              ((package_source.package_source_kaggle == 'both') | \
                               (package_source.package_source_kaggle == kaggle))\
                             ].merge(meta_package[(meta_package.meta_package_valid == True) & \
                                     ((meta_package.meta_package_data_size == 'both') | \
                                     (meta_package.meta_package_data_size == data_size))], \
                                     left_on  = 'meta_package_index', \
                                     right_on = 'meta_package_index', \
                                     how = 'inner') \
                             [['package_source_index', 'package_source_name', \
                               'package_source_short_name', 'package_source_code']]
pd_pk_import

,package_source_index,package_source_name,package_source_short_name,package_source_code
0,NP,numpy,np,None
1,PD,pandas,pd,None
2,SNS,seaborn,sns,None
3,OS,os,None,None
4,GC,gc,None,None
5,MPLPL,matplotlib.pyplot,plt,None
6,WARN,warnings,None,warnings.filterwarnings('ignore')
7,CPUN,None,None,os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
8,RE,re,None,None
9,MATH,math,None,None


In [18]:
package_source_type = 'partial'
data_size = 'small'

In [19]:
pd_pk_srce_from = package_source[(package_source.package_source_type == package_source_type)\
                                ].merge(meta_package[(meta_package.meta_package_valid == True) & \
                                         ((meta_package.meta_package_data_size == 'both') | \
                                         (meta_package.meta_package_data_size == data_size))], \
                                        left_on  = 'meta_package_index', \
                                        right_on = 'meta_package_index', \
                                        how = 'inner') \
                                [['package_source_index', 'package_source_name', \
                                  'package_source_short_name', 'package_source_code']] 
pd_pk_srce_from

,package_source_index,package_source_name,package_source_short_name,package_source_code
0,PDAT,pandas.api.types,None,None
1,ITT,itertools,None,None
2,JL,joblib,None,None
3,SCIP,scipy,None,None
4,SKL,sklearn,None,None
5,SKLB,sklearn.base,None,None
6,SKLE,sklearn.experimental,None,None
7,SKLI,sklearn.impute,None,None
8,SKLEN,sklearn.ensemble,None,None
9,SKLIN,sklearn.inspection,None,None


In [20]:
problem = 'regression'

In [21]:
pd_pk_from = package[((package.package_problem == 'None') | (package.package_problem == problem)) \
                    ].merge(pd_pk_srce_from, \
                            left_on='package_source_index', \
                            right_on='package_source_index',\
                            how='inner') \
                     .merge(meta_package[(meta_package.meta_package_valid == True) & \
                                         ((meta_package.meta_package_data_size == 'both') | \
                                         (meta_package.meta_package_data_size == data_size))], \
                            left_on  = 'meta_package_index', \
                            right_on = 'meta_package_index', \
                            how = 'inner') \
                [['package_source_index', 'package_source_name', 'package_name']].drop_duplicates()
pd_pk_from

,package_source_index,package_source_name,package_name
0,SKLB,sklearn.base,BaseEstimator
1,SKLB,sklearn.base,TransformerMixin
2,SKLNE,sklearn.neighbors,LocalOutlierFactor
3,SKLE,sklearn.experimental,enable_iterative_imputer
4,SKLI,sklearn.impute,SimpleImputer
5,SKLI,sklearn.impute,IterativeImputer
6,SKLIN,sklearn.inspection,permutation_importance
7,SKLGPK,sklearn.gaussian_process.kernels,RBF
8,SKLGPK,sklearn.gaussian_process.kernels,WhiteKernel
9,SKLGPK,sklearn.gaussian_process.kernels,ConstantKernel


In [22]:
pd_level_0 = package[(package.package_type == 'estimator') & \
                     (package.package_problem == problem) & \
                     (package.meta_package_index != 'STACK') \
                    ].merge(pd_pk_from, \
                            left_on='package_source_index', \
                            right_on='package_source_index',\
                            how='inner') \
                     .merge(meta_package[(meta_package.meta_package_valid == True) & \
                                         ((meta_package.meta_package_data_size == 'both') | \
                                         (meta_package.meta_package_data_size == data_size))], \
                            left_on  = 'meta_package_index', \
                            right_on = 'meta_package_index', \
                            how = 'inner') \
                    [[ 'package_index', 'package_code', 'meta_package_tree']].drop_duplicates()
pd_level_0

,package_index,package_code,meta_package_tree
0,GPRL,GaussianProcessRegressor(kernel = ConstantKern...,0.0
1,GPRR,GaussianProcessRegressor(kernel = ConstantKern...,0.0
2,GPRQ,GaussianProcessRegressor(kernel = ConstantKern...,0.0
3,DTRF,DecisionTreeRegressor(criterion='friedman_mse'...,1.0
4,DTRA,DecisionTreeRegressor(criterion='absolute_erro...,1.0
5,DTRP,"DecisionTreeRegressor(criterion='poisson', ran...",1.0
6,RFRS,RandomForestRegressor(criterion='squared_error...,1.0
10,RFRA,RandomForestRegressor(criterion='absolute_erro...,1.0
14,RFRP,"RandomForestRegressor(criterion='poisson', n_e...",1.0
18,ABR,AdaBoostRegressor(random_state = random_state),1.0


In [23]:
pd_tree = package[(package.package_type == 'estimator') & \
                  (package.package_problem == problem) & \
                  (package.meta_package_index != 'STACK') \
                 ].merge(pd_pk_from, \
                         left_on='package_source_index', \
                         right_on='package_source_index',\
                         how='inner') \
                  .merge(meta_package[(meta_package.meta_package_tree == True) & \
                                      (meta_package.meta_package_valid == True) & \
                                      ((meta_package.meta_package_data_size == 'both') | \
                                      (meta_package.meta_package_data_size == data_size))], \
                         left_on  = 'meta_package_index', \
                         right_on = 'meta_package_index', \
                         how = 'inner') \
                  [[ 'package_index', 'package_code']].drop_duplicates()
pd_tree

,package_index,package_code
0,DTRF,DecisionTreeRegressor(criterion='friedman_mse'...
1,DTRA,DecisionTreeRegressor(criterion='absolute_erro...
2,DTRP,"DecisionTreeRegressor(criterion='poisson', ran..."
3,RFRS,RandomForestRegressor(criterion='squared_error...
7,RFRA,RandomForestRegressor(criterion='absolute_erro...
11,RFRP,"RandomForestRegressor(criterion='poisson', n_e..."
15,ABR,AdaBoostRegressor(random_state = random_state)
19,HGBR,HistGradientBoostingRegressor(early_stopping=T...
23,XGBR,"XGBRegressor(max_depth=5, n_estimators=20, ver..."


In [24]:
pd_document = document[((document.document_problem == 'both') | (document.document_problem == problem)) & \
                       ((document.document_data_size == 'both') | (document.document_data_size == data_size)) & \
                       ((document.document_stacking == 'both') | \
                       (document.document_stacking == \
                        meta_package[meta_package.meta_package_index == 'STACK']\
                        ['meta_package_valid'].tolist()[0]\
                       )) & \
                       ((document.document_keras == 'both') | \
                       (document.document_keras == \
                        meta_package[meta_package.meta_package_index == 'KER']\
                        ['meta_package_valid'].tolist()[0]\
                       )) & \
                       ((document.document_xgb == 'both') | \
                       (document.document_xgb == \
                        meta_package[meta_package.meta_package_index == 'XGB']\
                        ['meta_package_valid'].tolist()[0]\
                       )) & \
                       ((document.document_pipeline == 'both') | \
                       (document.document_pipeline == \
                        meta_package[meta_package.meta_package_index == 'PIP']\
                        ['meta_package_valid'].tolist()[0]\
                       )) & \
                       ((document.document_yb == 'both') | \
                       (document.document_yb == \
                        meta_package[meta_package.meta_package_index == 'YB']\
                        ['meta_package_valid'].tolist()[0]\
                       )) \
                      ].merge(meta_package[meta_package.meta_package_valid == True], \
                                           left_on  = 'meta_package_index', \
                                           right_on = 'meta_package_index', \
                                           how = 'inner') \
                      [['sort_key', 'title', 'text', 'code']].sort_values('sort_key') \
                      [['title', 'text', 'code']].drop_duplicates()

pd_document

,title,text,code
0,#,EDA & Modelling,"""random_state = "" + str(random_state)"
1,#,Exploratory Data Analysis,None
2,##,File and parameters loading,"""problem_type = '"" + str(problem_type) + ""'"""
3,,None,"""data_size = '"" + str(data_size) + ""'"""
38,,None,"""df = pd.read_csv('"" + str(file) + ""')"""
4,,None,"""target_col = '"" + str(target_col) + ""'"""
5,###,Thresholds & other parameters,"""threshold_NaN = "" + str(threshold_NaN)"
6,,None,"""threshold_cat = "" + str(threshold_cat)"
7,,None,"""threshold_Z = "" + str(threshold_Z)"
8,,None,"""test_size = "" + str(test_size)"
